In [1]:
# First cell: Setup and installations
!pip install unsloth
!pip install transformers
!pip install trl
!pip install rouge
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.8/188.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.1/107.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8

In [2]:
# Second cell: Imports and configurations
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from rouge import Rouge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import random
import numpy as np
import os

# Set up paths for Kaggle
OUTPUT_DIR = "/kaggle/working/finqa_finetuned"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Third cell: Data preparation functions
finqa_prompt = """Below contains texts before table (pre-text), text after the table (post-text) and the table itself with a question that you must answer by providing only the mathematical or logical operations needed to solve it along with the result.
### Pre-text:
{}
### Table:
{}
### Post-text:
{}
### Question:
{}
### Response:
{}"""

def format_example(example):
    """Format a single example from FinQA dataset with simplified format"""
    pre_text = ' '.join(example['pre_text']) if example['pre_text'] else ""
    post_text = ' '.join(example['post_text']) if example['post_text'] else ""
    table = "\n".join(" | ".join(str(cell) for cell in row) for row in example['table']) if example['table'] else ""
    
    # Format output with operation and result on one line
    output = f"{example['program_re']} = {example['final_result']}"
    
    # Format using the simplified template
    text = finqa_prompt.format(
        pre_text,
        table,
        post_text,
        example['question'],
        output
    ) + tokenizer.eos_token
    
    # Return as a dictionary with 'text' key
    return {"text": text}

def prepare_dataset():
    """Prepare FinQA dataset for fine-tuning"""
    from datasets import Dataset
    
    # Load dataset
    dataset = load_dataset("ibm/finqa", split="train", trust_remote_code=True)
    
    # Filter for valid examples
    train_data = [
        ex for ex in dataset
        if ex['program_re'] and ex['final_result'] is not None  # Changed from expanded_program_re
    ]
    
    # Format examples
    formatted_examples = [format_example(ex) for ex in train_data]
    
    # Convert to Dataset object
    return Dataset.from_list(formatted_examples)

In [4]:
# Fourth cell: Model setup and training
# Load model and tokenizer
model_name = "unsloth/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora = False,
    loftq_config = None
)

tokenizer_config.json:   0%|          | 0.00/7.36k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
# Fifth cell: Training
# Prepare data
train_data = prepare_dataset()

# Prepare training arguments
train_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    # max_steps=100,
    num_train_epochs = 1, # Set this for 1 full training run.
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    # max_grad_norm=0.3,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir=OUTPUT_DIR,
    report_to="none",
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc = 2,
    packing = False,
    tokenizer=tokenizer,
    args=train_args,
)

# Print initial memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Train
print("\nStarting training...")
trainer_stats = trainer.train()

# Print final stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
print(f"\nTraining completed in {trainer_stats.metrics['train_runtime']} seconds")
print(f"Peak memory usage: {used_memory} GB")
print(f"Memory used for training: {used_memory_for_lora} GB")

README.md:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

finqa.py:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

Truncating train dataset (num_proc=2):   0%|          | 0/6251 [00:00<?, ? examples/s]

GPU = Tesla T4. Max memory = 14.741 GB.
6.879 GB of memory reserved.

Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6,251 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 781
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.689700
2,1.645000
3,1.819000
4,1.767700
5,1.551300
6,1.722200
7,1.591000
8,1.613600
9,1.319100
10,1.331700



Training completed in 24348.7244 seconds
Peak memory usage: 9.014 GB
Memory used for training: 2.135 GB


In [6]:
# Sixth cell: Save the model
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

model.push_to_hub("n3Er/qwen2.5-7b-instruct-finqa-ht", token = "your_private_token") # Online saving
tokenizer.push_to_hub("n3Er/qwen2.5-7b-instruct-finqa-ht", token = "your_private_token") # Online saving


FastLanguageModel.for_inference(model)

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/162M [00:00<?, ?B/s]

Saved model to https://huggingface.co/n3Er/qwen2.5-7b-instruct-finqa-ht


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-1): 2 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [7]:
# Seventh cell: Evaluation functions
def format_test_input(example):
    """Format a test example for inference"""
    pre_text = ' '.join(example['pre_text']) if example['pre_text'] else ""
    post_text = ' '.join(example['post_text']) if example['post_text'] else ""
    table = "\n".join(" | ".join(str(cell) for cell in row) for row in example['table']) if example['table'] else ""
    
    return finqa_prompt.format(
        pre_text,
        table,
        post_text,
        example['question'],
        ""  # Empty response for generation
    )

def extract_operations_result(response):
    """Extract operations and result from model response"""
    # Get only the Response section
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    
    # Look for the equation format: operations = result
    if "=" in response:
        parts = response.split("=", 1)
        operations = parts[0].strip()
        result = parts[1].strip() if len(parts) > 1 else None
        return operations, result
    
    return None, None

def evaluate_sample(model, tokenizer, example):
    """Evaluate model on a single example"""
    # Format input
    input_text = format_test_input(example)
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda")
    
    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        temperature=1e-10,
        # top_p=0.9,
        # do_sample=True,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        # repetition_penalty=1.2
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract generated and expected parts
    generated_ops, generated_result = extract_operations_result(response)
    expected_ops = example['program_re']  # Changed from expanded_program_re
    expected_result = str(example['final_result'])
    
    # Calculate metrics
    rouge = Rouge()
    try:
        rouge_scores = rouge.get_scores(
            generated_ops or "",
            expected_ops
        )[0]
    except:
        rouge_scores = {
            'rouge-1': {'f': 0.0},
            'rouge-2': {'f': 0.0},
            'rouge-l': {'f': 0.0}
        }
    
    # Calculate Cosine similarity
    vectorizer = TfidfVectorizer()
    try:
        tfidf_matrix = vectorizer.fit_transform([
            generated_ops or "",
            expected_ops
        ])
        cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    except:
        cosine_sim = 0.0
    
    return {
        'question': example['question'],
        'expected_equation': f"{expected_ops} = {expected_result}",
        'generated_equation': f"{generated_ops} = {generated_result}" if generated_ops else None,
        'rouge1_f': rouge_scores['rouge-1']['f'],
        'rouge2_f': rouge_scores['rouge-2']['f'],
        'rougeL_f': rouge_scores['rouge-l']['f'],
        'cosine_similarity': cosine_sim,
        'raw_response': response
    }

In [8]:
# Eighth cell: Run evaluation
def evaluate_model(num_samples=50):
    """Evaluate model on test set samples"""
    print("Loading test dataset...")
    dataset = load_dataset("ibm/finqa", split="test", trust_remote_code=True)
    
    # Filter valid examples
    valid_examples = [
        ex for ex in dataset 
        if ex['program_re'] and ex['final_result'] is not None
    ]
    
    # Select random samples
    test_samples = random.sample(valid_examples, min(num_samples, len(valid_examples)))
    
    # Evaluate samples
    results = []
    total_samples = len(test_samples)
    print(f"\nEvaluating {total_samples} samples...")
    
    for idx, sample in enumerate(test_samples, 1):
        print(f"\nSample {idx}/{total_samples}")
        result = evaluate_sample(model, tokenizer, sample)
        results.append(result)
        
        # Print individual result
        print(f"Question: {result['question']}")
        print(f"Expected: {result['expected_equation']}")
        print(f"Generated: {result['generated_equation']}")
        print(f"ROUGE-1 F1: {result['rouge1_f']:.3f}")
        print(f"ROUGE-2 F1: {result['rouge2_f']:.3f}")
        print(f"ROUGE-L F1: {result['rougeL_f']:.3f}")
        print(f"Cosine Similarity: {result['cosine_similarity']:.3f}")
    
    # Calculate average metrics
    avg_metrics = {
        'rouge1_f': np.mean([r['rouge1_f'] for r in results]),
        'rouge2_f': np.mean([r['rouge2_f'] for r in results]),
        'rougeL_f': np.mean([r['rougeL_f'] for r in results]),
        'cosine_similarity': np.mean([r['cosine_similarity'] for r in results])
    }
    
    print("\nOverall Results:")
    print("=" * 50)
    print(f"Average ROUGE-1 F1: {avg_metrics['rouge1_f']:.3f}")
    print(f"Average ROUGE-2 F1: {avg_metrics['rouge2_f']:.3f}")
    print(f"Average ROUGE-L F1: {avg_metrics['rougeL_f']:.3f}")
    print(f"Average Cosine Similarity: {avg_metrics['cosine_similarity']:.3f}")
    
    return results, avg_metrics

In [9]:
# Ninth cell: Execute evaluation
print("Starting model evaluation...")
results, metrics = evaluate_model(num_samples=50)

Starting model evaluation...
Loading test dataset...

Evaluating 50 samples...

Sample 1/50
Question: what is the total in millions of expected cash outflow to satisfy contractual obligations and commitments as of december 31 , 2007?
Expected: table_sum(total, none) = 28809
Generated: add(619, 1782), add(#0, 10527), add(#1, 8026), add(#2, 3828), add(#3, 3465), add(#4, 562) = 
ROUGE-1 F1: 0.000
ROUGE-2 F1: 0.000
ROUGE-L F1: 0.000
Cosine Similarity: 0.000

Sample 2/50
Question: what is the net change in aon 2019s unpaid restructuring liabilities during 2007?
Expected: add(38, -110), add(#0, 1) = -71
Generated: subtract(63, 134) = -71
ROUGE-1 F1: 0.000
ROUGE-2 F1: 0.000
ROUGE-L F1: 0.000
Cosine Similarity: 0.000

Sample 3/50
Question: what is the percent change of the amount of collateral held for indemnified securities between 2006 and 2007?
Expected: subtract(572.93, 527.37), divide(#0, 527.37) = 8.6%
Generated: subtract(572.93, 527.37), divide(#0, 527.37) = 8.6%
ROUGE-1 F1: 1.000
ROUGE

Unsloth: Input IDs of length 2283 > the model's max sequence length of 2048.
We shall truncate it ourselves. It's imperative if you correct this issue first.


Question: what was the total impairment costs recorded from 2003 to 2005 in millions
Expected: add(0.6, 0.5), add(#0, 4.7) = 5.8
Generated: add(0.5, 0.6), add(#0, 4.7), add(#1, 0.6), add(#2, 0.2) = 6.6
ROUGE-1 F1: 0.471
ROUGE-2 F1: 0.118
ROUGE-L F1: 0.471
Cosine Similarity: 1.000

Sample 41/50


RuntimeError: The size of tensor a (2048) must match the size of tensor b (2283) at non-singleton dimension 2